# Example to Read / Write to Microsoft SQL Server with Spark

Documentation: https://docs.microsoft.com/en-us/sql/connect/spark/connector?view=sql-server-ver15

An Improvement on the base JDBC connector.


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
# SQLSERVRER CONFIGURATION
server = "mssql"
server_str = f"jdbc:sqlserver://{server}:1433"
database_name = "master" 
url = f"{server_str};databaseName={database_name}"
username = "sa"
password = "SU2Orange!" 

In [3]:
# Spark init
spark = SparkSession \
    .builder \
    .master("local") \
    .appName('jupyter-pyspark') \
      .config("spark.jars.packages","com.microsoft.azure:spark-mssql-connector_2.12_3.0:1.0.0-alpha,com.microsoft.sqlserver:mssql-jdbc:9.4.0.jre11")\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.microsoft.azure#spark-mssql-connector_2.12_3.0 added as a dependency
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6f435795-543a-4d5d-8a11-34e82d60b429;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.microsoft.azure#spark-mssql-connector_2.12_3.0;1.0.0-alpha in central
	found com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre11 in central
:: resolution report :: resolve 97ms :: artifacts dl 2ms
	:: modules in use:
	com.microsoft.azure#spark-mssql-connector_2.12_3.0;1.0.0-alpha from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre11 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-6f435795-543a-4d5d-8a11-34e82d60b429
	confs: [default]
	0 artifacts copied, 2 already retrieved (0kB/3ms)
21/11/16 23:59:11 WARN Nati

In [4]:
# read local data
df = spark.read.option("multiline","true").json("/home/jovyan/datasets/json-samples/stocks.json")
df.toPandas()

,price,symbol
0,126.82,AAPL
1,3098.12,AMZN
2,251.11,FB
3,1725.05,GOOG
4,128.39,IBM
5,212.55,MSFT
6,78.00,NET
7,497.00,NFLX
8,823.80,TSLA
9,45.11,TWTR


In [5]:
# Write create the table on write:
print(url)
df.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("url", url) \
    .option("dbtable", "stocks") \
    .option("user", username) \
    .option("password", password) \
    .save()

jdbc:sqlserver://mssql:1433;databaseName=master


Py4JJavaError: An error occurred while calling o55.save.
: java.sql.SQLException: No suitable driver
	at java.sql/java.sql.DriverManager.getDriver(DriverManager.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$2(JDBCOptions.scala:108)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:108)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:217)
	at com.microsoft.sqlserver.jdbc.spark.SQLServerBulkJdbcOptions.<init>(SQLServerBulkJdbcOptions.scala:25)
	at com.microsoft.sqlserver.jdbc.spark.SQLServerBulkJdbcOptions.<init>(SQLServerBulkJdbcOptions.scala:27)
	at com.microsoft.sqlserver.jdbc.spark.DefaultSource.createRelation(DefaultSource.scala:55)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:46)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec

In [19]:
# read back from Mongo
df1 = spark.read.format("mongo") \
    .option("database","example") \
    .option("collection","stocks") \
    .load()
df1.toPandas()

IllegalArgumentException: requirement failed: Missing 'uri' property from options